In [5]:
import pandas as pd
import numpy as np
import tqdm
import scipy.stats as sts

In [7]:
features = pd.read_csv('./large_data/output_data/final_dfdiouuy.csv')
clients = pd.read_csv('./large_data/raw_data/clients.csv', index_col=0)

In [11]:
poor = pd.read_csv('./large_data/raw_data/poor_raw.csv')
rich = pd.read_csv('./large_data/raw_data/rich_raw.csv')

total_df = pd.concat([poor, rich])

del poor
del rich

In [12]:
total_df = total_df[[
    'client_id', 'transaction_id',
    'transaction_datetime', 'purchase_sum',
    'age', 'gender',
    'treatment_flg', 'city_type',
]].drop_duplicates()

In [13]:
total_df['transaction_datetime'] = pd.to_datetime(total_df['transaction_datetime']).dt.date

**разбиваем фичу с полом**

In [14]:
total_df['female'] = (total_df.gender == 'F').astype(int)
total_df['male'] = (total_df.gender == 'M').astype(int)

total_df.drop(columns=['gender'], inplace=True)

**процессинг тагрета**

In [15]:
def modify_products(row):
    ''' Функция для зашумления людей из богатого города, на которых разослали коммуникацию'''
    
    if (row['treatment_flg'] == 1) & (row['city_type'] == 1):
        # Генерируем экспоненциальную случайную величину с матожиданием row['age']
        exp_value = np.random.exponential(5*abs(row['age']))
        return row['purchase_sum'] + exp_value
    return row['purchase_sum']

# Применяем функцию по строкам DataFrame
total_df['purchase_sum'] = total_df.apply(modify_products, axis=1)

In [16]:
total_df = total_df.groupby(['client_id', 'transaction_datetime'], as_index=False).agg({
    'purchase_sum':'sum',
    'treatment_flg':'mean',
    'city_type':'mean',
    'female':'mean',
    'male':'mean',
})
total_df.rename(columns={'transaction_datetime':'transaction_date'}, inplace=True)

In [17]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7145662 entries, 0 to 7145661
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   client_id         object 
 1   transaction_date  object 
 2   purchase_sum      float64
 3   treatment_flg     float64
 4   city_type         float64
 5   female            float64
 6   male              float64
dtypes: float64(5), object(2)
memory usage: 381.6+ MB


In [18]:
res_data = pd.concat([features, total_df.drop(columns=['client_id','client_id'])],axis=1)

**собираем красивую панель**

In [19]:
res_data = res_data[[
    'client_id','transaction_date',
    'city_type','treatment_flg',
    'age', 'female', 'male',
    'purchase_sum','avg_product_quantity', 
    'total_products', 'ch_avg_purchase',
    'unique_segments', 'unique_brands', 
    'alcohol_percentage','own_trademark_percentage',
]]

In [20]:
res_data.to_csv('./large_data/intermediate_data/panel_data.csv')